# Exp 04: lgb agrupando por periodo-product_id

Usando distribución tweedy y max_bins=500 y optimizacion bayesiana

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

##### Merge: periodos + clientes + productos

In [4]:
df = pd.read_csv("../../data/preprocessed/base.csv", sep=',')
df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")


periodos = pd.date_range(start=df['periodo_dt'].min(), end=df['periodo_dt'].max(), freq="MS")
productos = df['product_id'].unique()


idx = pd.MultiIndex.from_product([productos, periodos], names=['product_id', 'periodo'])
completo = idx.to_frame(index=False)
completo["periodo"] = completo["periodo"].dt.strftime("%Y%m").astype(int)

del periodos, productos, 
gc.collect()
completo

,product_id,periodo
0,20524,201701
1,20524,201702
2,20524,201703
3,20524,201704
4,20524,201705
...,...,...
44383,20770,201908
44384,20770,201909
44385,20770,201910
44386,20770,201911


##### Cruzamos con productos

In [5]:
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
completo = completo.merge(productos, how='left', on="product_id")
del productos
gc.collect()

0

##### Cruzamos con stock

In [6]:
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')
stocks = stocks.groupby(by=["periodo", "product_id"]).agg({"stock_final": "sum"}).reset_index()
completo = completo.merge(stocks, how='left', on=['periodo', 'product_id'])
del stocks
gc.collect()
completo.head()

,product_id,periodo,cat1,cat2,cat3,brand,sku_size,stock_final
0,20524,201701,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,20524,201702,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,20524,201703,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,20524,201704,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,20524,201705,HC,VAJILLA,Cristalino,Importado,500.0,NaN


##### Cruzamos con ventas

In [7]:
sellin = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
# Agrupar ventas por periodo, cliente y producto
dt = sellin.groupby(by=["periodo","product_id"]).agg({"tn":"sum"}).reset_index()
df_completo = completo.merge(dt, how='left', on=['periodo', 'product_id'])
df_completo['tn'] = df_completo['tn'].fillna(0)
del sellin, dt, completo
gc.collect()
df_completo

,product_id,periodo,cat1,cat2,cat3,brand,sku_size,stock_final,tn
0,20524,201701,HC,VAJILLA,Cristalino,Importado,500.0,NaN,6.48085
1,20524,201702,HC,VAJILLA,Cristalino,Importado,500.0,NaN,3.99755
2,20524,201703,HC,VAJILLA,Cristalino,Importado,500.0,NaN,7.14711
3,20524,201704,HC,VAJILLA,Cristalino,Importado,500.0,NaN,6.82163
4,20524,201705,HC,VAJILLA,Cristalino,Importado,500.0,NaN,9.25949
...,...,...,...,...,...,...,...,...,...
44383,20770,201908,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25.0,NaN,0.00000
44384,20770,201909,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25.0,NaN,0.00000
44385,20770,201910,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25.0,NaN,0.00000
44386,20770,201911,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25.0,NaN,0.00000


##### Target

In [10]:
# Asegurarte de tener 'periodo_dt' (datetime) en completo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear DataFrame auxiliar con tn como target y fecha adelantada
ventas_futuras = df_completo[['periodo_dt', 'product_id', 'tn']].copy()
ventas_futuras['periodo_target_dt'] = ventas_futuras['periodo_dt'] - pd.DateOffset(months=2)
ventas_futuras = ventas_futuras.rename(columns={'tn': 'target'})

# Merge con completo usando periodo adelantado
df_completo = df_completo.merge(
    ventas_futuras[['periodo_target_dt', 'product_id', 'target']],
    how='left',
    left_on=['periodo_dt', 'product_id'],
    right_on=['periodo_target_dt', 'product_id']
)

# Eliminar columna auxiliar
df_completo = df_completo.drop(columns=['periodo_target_dt'])
del ventas_futuras
gc.collect()
print(f"✅ Target generado. Filas con target no nulo: {df_completo['target'].notna().sum()}")

✅ Target generado. Filas con target no nulo: 41922


In [11]:
df_completo

,product_id,periodo,cat1,cat2,cat3,brand,sku_size,stock_final,tn,periodo_dt,target_x,target_y,target
0,20524,201701,HC,VAJILLA,Cristalino,Importado,500.0,NaN,6.48085,2017-01-01,7.14711,7.14711,7.14711
1,20524,201702,HC,VAJILLA,Cristalino,Importado,500.0,NaN,3.99755,2017-02-01,6.82163,6.82163,6.82163
2,20524,201703,HC,VAJILLA,Cristalino,Importado,500.0,NaN,7.14711,2017-03-01,9.25949,9.25949,9.25949
3,20524,201704,HC,VAJILLA,Cristalino,Importado,500.0,NaN,6.82163,2017-04-01,7.04113,7.04113,7.04113
4,20524,201705,HC,VAJILLA,Cristalino,Importado,500.0,NaN,9.25949,2017-05-01,5.92819,5.92819,5.92819
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44383,20770,201908,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25.0,NaN,0.00000,2019-08-01,0.00000,0.00000,0.00000
44384,20770,201909,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25.0,NaN,0.00000,2019-09-01,0.00000,0.00000,0.00000
44385,20770,201910,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25.0,NaN,0.00000,2019-10-01,3.18500,3.18500,3.18500
44386,20770,201911,HC,PROFESIONAL,LV ROPA POLVO,INDUSTRIAL,25.0,NaN,0.00000,2019-11-01,NaN,NaN,NaN


##### Verifico las NaN en el target: Existen porque hay clientes que solo compraron 2 veces.

In [12]:
nan_count = df_completo['target'].isna().sum()
print(f"🔍 Total de NaN en target: {nan_count}")
del nan_count
gc.collect()

🔍 Total de NaN en target: 2466


0

##### Generación de IDs

In [13]:
df_completo = df_completo.sort_values(['periodo', 'product_id'])
df_completo['id'] = df_completo.groupby(['product_id']).cumcount() + 1

##### Periodo 

In [14]:
df_completo["periodo_dt"] = pd.to_datetime(df_completo["periodo"].astype(str), format="%Y%m")

##### Eliminar productos que no nacieron

In [15]:
nacimiento_producto = df.groupby("product_id")["periodo_dt"].agg(["min"]).reset_index()
# Renombrar columna max a muerte_cliente_dt
nacimiento_producto = nacimiento_producto.rename(columns={'min': 'nacimiento_producto'})


# Unir con df_final para traer fecha de muerte del cliente
df_completo = df_completo.merge(nacimiento_producto, on='product_id', how='left')

# Filtrar filas donde periodo_dt > muerte_cliente_dt
df_completo = df_completo[df_completo['periodo_dt'] >= df_completo['nacimiento_producto']]

# Opcional: eliminar columna auxiliar
# df_final = df_final.drop(columns=['muerte_cliente_dt'])
del nacimiento_producto
gc.collect()
print(f"✅ Dataset filtrado con {len(df_completo):,} filas.")

✅ Dataset filtrado con 35,888 filas.


##### Correlograma

In [16]:
# Supongamos que tu DataFrame se llama df
cor_matrix = df_completo.corr(numeric_only=True)

# Tomamos solo la parte superior de la matriz (sin la diagonal)
upper = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(bool))

# Filtramos correlaciones fuertes (valor absoluto mayor a 0.7)
high_corr = upper.stack().reset_index()
high_corr.columns = ['Variable 1', 'Variable 2', 'Correlación']
high_corr_filtrada = high_corr[high_corr['Correlación'].abs() > 0.7]

print(high_corr_filtrada)

del high_corr_filtrada, cor_matrix, upper, high_corr
gc.collect()

   Variable 1 Variable 2  Correlación
14    periodo         id     0.955974
26         tn   target_x     0.930409
27         tn   target_y     0.930409
28         tn     target     0.930409
30   target_x   target_y     1.000000
31   target_x     target     1.000000
33   target_y     target     1.000000


0

##### Elimino variables muy correlacionadas

In [ ]:
df_completo.drop(columns=['periodo'], inplace=True)

##### Extracción de componentes temporales

In [17]:
df_completo['year'] = df_completo['periodo_dt'].dt.year
df_completo['month'] = df_completo['periodo_dt'].dt.month
# Variables dummy estacionales
df_completo['quarter'] = df_completo['periodo_dt'].dt.quarter
df_completo['semester'] = np.where(df_completo['month'] <= 6, 1, 2)
# Efectos de fin de año
df_completo['year_end'] = np.where(df_completo['month'].isin([11, 12]), 1, 0)
df_completo['year_start'] = np.where(df_completo['month'].isin([1, 2]), 1, 0)
# Indicadores estacionales
df_completo['season'] = df_completo['month'] % 12 // 3 + 1  # 1:Invierno, 2:Primavera, etc.
# Variables cíclicas (para capturar patrones estacionales)
df_completo['month_sin'] = np.sin(2 * np.pi * df_completo['month']/12)
df_completo['month_cos'] = np.cos(2 * np.pi * df_completo['month']/12)

#####  Lags, diferencias, medias móviles y otras yerbas

In [18]:
# Ordenamos por fecha para asegurar consistencia
df_completo = df_completo.sort_values('periodo_dt')

## 1. Lags (rezagos) de 1 a 12 meses
for i in range(1, 15):
    df_completo[f'lag_{i}'] = df_completo['tn'].shift(i)

## 2. Diferencias (deltas) - cambio respecto al mes anterior
for i in range(1, 13):
    df_completo[f'delta_{i}'] = df_completo['tn'].diff(i)

## 3. Diferencias porcentuales
for i in range(1, 13):
    df_completo[f'pct_change_{i}'] = df_completo['tn'].pct_change(i)

## 4. Medias móviles (promedios móviles)
windows = [2, 3, 6, 9, 12]  # También puedes incluir [2,4,5,7] según necesidad
for w in windows:
    df_completo[f'rolling_mean_{w}'] = df_completo['tn'].rolling(window=w, min_periods=1).mean()
    df_completo[f'rolling_std_{w}'] = df_completo['tn'].rolling(window=w, min_periods=1).std()
    df_completo[f'rolling_min_{w}'] = df_completo['tn'].rolling(window=w, min_periods=1).min()
    df_completo[f'rolling_max_{w}'] = df_completo['tn'].rolling(window=w, min_periods=1).max()
    df_completo[f'rolling_median_{w}'] = df_completo['tn'].rolling(window=w, min_periods=1).median()

## 5. Características de tendencia y estacionalidad
df_completo['expanding_mean'] = df_completo['tn'].expanding().mean()
df_completo['cumulative_sum'] = df_completo['tn'].cumsum()

## 6. Características de diferencia estacional (12 meses para datos mensuales)
df_completo['seasonal_diff_12'] = df_completo['tn'].diff(12)

## 7. Estadísticas anuales comparativas
df_completo['vs_prev_year'] = df_completo['tn'] / df_completo['lag_12'] - 1  # Crecimiento interanual

## 8. Componentes de descomposición (simplificada)
# Tendencia (usando media móvil de 12 meses)
df_completo['trend'] = df_completo['tn'].rolling(window=12, min_periods=1).mean()
# Estacionalidad (diferencia entre valor real y tendencia)
df_completo['seasonality'] = df_completo['tn'] - df_completo['trend']

## 9. Variables booleanas para eventos especiales
df_completo['new_high'] = (df_completo['tn'] == df_completo['rolling_max_12']).astype(int)
df_completo['new_low'] = (df_completo['tn'] == df_completo['rolling_min_12']).astype(int)

## 10. Características de aceleración/deceleración
df_completo['acceleration'] = df_completo['delta_1'].diff(1)  # Cambio en la tasa de cambio

##### Estadísticas de Ventana Dinámica

In [19]:
# Medias móviles exponenciales
df_completo['ewm_alpha_0.3'] = df_completo['tn'].ewm(alpha=0.3, adjust=False).mean()
df_completo['ewm_alpha_0.5'] = df_completo['tn'].ewm(alpha=0.5, adjust=False).mean()

# Medias móviles centradas
df_completo['rolling_center_mean_3'] = df_completo['tn'].rolling(window=3, center=True).mean()

# Sumas acumuladas por año
df_completo['ytd_sum'] = df_completo.groupby(df_completo['periodo_dt'].dt.year)['tn'].cumsum()

##### Características de Tendencia y Ciclo

In [20]:
# Modelado de tendencia polinomial
df_completo['time_index'] = range(len(df_completo))
df_completo['trend_linear'] = np.poly1d(np.polyfit(df_completo['time_index'], df_completo['tn'], 1))(df_completo['time_index'])
df_completo['trend_quadratic'] = np.poly1d(np.polyfit(df_completo['time_index'], df_completo['tn'], 2))(df_completo['time_index'])

# Residuales de tendencia
df_completo['residual_trend'] = df_completo['tn'] - df_completo['trend_linear']

##### Características de Cambio de Régimen

In [21]:
# Z-Score respecto a ventana móvil
df_completo['zscore_6'] = (df_completo['tn'] - df_completo['rolling_mean_6']) / df_completo['rolling_std_6']

# Detección de outliers
df_completo['is_outlier_3sigma'] = np.where(np.abs(df_completo['zscore_6']) > 3, 1, 0)

# Cambios bruscos (spikes)
df_completo['spike_up'] = np.where(df_completo['delta_1'] > df_completo['rolling_std_3'], 1, 0)
df_completo['spike_down'] = np.where(df_completo['delta_1'] < -df_completo['rolling_std_3'], 1, 0)

##### Características de Patrones Temporales

In [22]:
# Autocorrelaciones parciales
from statsmodels.tsa.stattools import pacf
pacf_values = pacf(df_completo['tn'].dropna(), nlags=12)
for i in range(1, 6):
    df_completo[f'pacf_{i}'] = df_completo['tn'].shift(i) * pacf_values[i]

# Estacionalidad múltiple (si hay patrones semestrales)
df_completo['semester_mean'] = df_completo.groupby(['year', 'semester'])['tn'].transform('mean')

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\1097231424.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo[f'pacf_{i}'] = df_completo['tn'].shift(i) * pacf_values[i]
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\1097231424.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo[f'pacf_{i}'] = df_completo['tn'].shift(i) * pacf_values[i]
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\1097231424.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

##### Características de Forecast Ingenieriles

In [23]:
# Método ingenuo (último valor)
df_completo['naive_forecast'] = df_completo['tn'].shift(1)

# Seasonal naive (valor del mismo período año anterior)
df_completo['seasonal_naive'] = df_completo['tn'].shift(12)

# Promedio móvil como forecast
df_completo['ma_forecast_3'] = df_completo['rolling_mean_3'].shift(1)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\732690613.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['naive_forecast'] = df_completo['tn'].shift(1)
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\732690613.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['seasonal_naive'] = df_completo['tn'].shift(12)
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\732690613.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

##### Características de Decomposición Temporal

In [24]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Descomposición clásica (additiva o multiplicativa)
result = seasonal_decompose(df_completo['tn'].dropna(), model='additive', period=12)
df_completo['trend_decomposed'] = result.trend
df_completo['seasonal_decomposed'] = result.seasonal
df_completo['residual_decomposed'] = result.resid

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\160459473.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['trend_decomposed'] = result.trend
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\160459473.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['seasonal_decomposed'] = result.seasonal
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\160459473.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

##### Características de Ventanas Asimétricas

In [25]:
# Mejor mes histórico
df_completo['best_month_rank'] = df_completo.groupby('month')['tn'].rank(ascending=False)

# Comparación con mismo mes año anterior
df_completo['vs_last_year_same_month'] = df_completo['tn'] / df_completo['lag_12'] - 1

# Acumulado últimos 3 vs mismos 3 meses año anterior
df_completo['last3_vs_ly3'] = (df_completo['tn'] + df_completo['lag_1'] + df_completo['lag_2']) / (df_completo['lag_12'] + df_completo['lag_13'] + df_completo['lag_14']) - 1

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3785039539.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['best_month_rank'] = df_completo.groupby('month')['tn'].rank(ascending=False)
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3785039539.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['vs_last_year_same_month'] = df_completo['tn'] / df_completo['lag_12'] - 1
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3785039539.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is 

##### Transformaciones Matemáticas

In [26]:
from scipy import stats
from scipy.special import boxcox1p

# Transformaciones clásicas
df_completo['log_tn'] = np.log1p(df_completo['tn'])
df_completo['sqrt_tn'] = np.sqrt(df_completo['tn'])

# Box-Cox (solo si tn > 0)
mask = df_completo['tn'] > 0
df_completo['boxcox_tn'] = np.nan
df_completo.loc[mask, 'boxcox_tn'], _ = stats.boxcox(df_completo.loc[mask, 'tn'])

# Diferenciación
df_completo['diff1_log'] = df_completo['log_tn'].diff(1)


C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2096757591.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['log_tn'] = np.log1p(df_completo['tn'])
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2096757591.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['sqrt_tn'] = np.sqrt(df_completo['tn'])
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2096757591.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

##### Características de Interacción

In [27]:
# Interacción entre tendencia y estacionalidad
df_completo['trend_season_interaction'] = df_completo['trend'] * df_completo['seasonal_decomposed']

# Interacción lags con estacionalidad
for i in [1, 2, 3, 12]:
    df_completo[f'lag_{i}_season_adj'] = df_completo[f'lag_{i}'] / df_completo['seasonal_decomposed']

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\1620691952.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['trend_season_interaction'] = df_completo['trend'] * df_completo['seasonal_decomposed']
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\1620691952.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo[f'lag_{i}_season_adj'] = df_completo[f'lag_{i}'] / df_completo['seasonal_decomposed']
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\1620691952.py:6: PerformanceWarning: DataFrame is highly 

##### Nuevos lags cruzados y acumulados

In [28]:
# Lags de rolling_mean
for i in [1, 2, 3]:
    df_completo[f'lag_mean6_{i}'] = df_completo['rolling_mean_6'].shift(i)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2160742807.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo[f'lag_mean6_{i}'] = df_completo['rolling_mean_6'].shift(i)
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2160742807.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo[f'lag_mean6_{i}'] = df_completo['rolling_mean_6'].shift(i)
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2160742807.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

##### Slope (pendiente) de la tendencia local

In [29]:
df_completo['trend_slope_6'] = df_completo['rolling_mean_6'].diff(1)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\800313449.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['trend_slope_6'] = df_completo['rolling_mean_6'].diff(1)


##### Cambios acumulados

In [30]:
df_completo['cumulative_change_3'] = df_completo['delta_1'] + df_completo['delta_2'] + df_completo['delta_3']

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2858654289.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['cumulative_change_3'] = df_completo['delta_1'] + df_completo['delta_2'] + df_completo['delta_3']


##### Razones entre ventanas

In [31]:
df_completo['mean_ratio_3_6'] = df_completo['rolling_mean_3'] / (df_completo['rolling_mean_6'] + 1e-6)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\448283787.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['mean_ratio_3_6'] = df_completo['rolling_mean_3'] / (df_completo['rolling_mean_6'] + 1e-6)


##### Coeficiente de variación

In [32]:
df_completo['cv_6'] = df_completo['rolling_std_6'] / (df_completo['rolling_mean_6'] + 1e-6)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3107567901.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['cv_6'] = df_completo['rolling_std_6'] / (df_completo['rolling_mean_6'] + 1e-6)


##### Ratio entre último valor y media móvil

In [33]:
df_completo['tn_vs_mean_3'] = df_completo['tn'] / (df_completo['rolling_mean_3'] + 1e-6)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3927802484.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['tn_vs_mean_3'] = df_completo['tn'] / (df_completo['rolling_mean_3'] + 1e-6)


##### Amplitud de la serie

In [34]:
df_completo['rolling_amplitude_6'] = df_completo['rolling_max_6'] - df_completo['rolling_min_6']

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3082303587.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['rolling_amplitude_6'] = df_completo['rolling_max_6'] - df_completo['rolling_min_6']


##### Count positivo/negativo en ventana

In [35]:
df_completo['positive_count_6'] = df_completo['tn'].rolling(6).apply(lambda x: (x > 0).sum())

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2184133645.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['positive_count_6'] = df_completo['tn'].rolling(6).apply(lambda x: (x > 0).sum())


##### Media de deltas

In [36]:
df_completo['delta_mean_3'] = df_completo['delta_1'] + df_completo['delta_2'] + df_completo['delta_3']

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\988223847.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['delta_mean_3'] = df_completo['delta_1'] + df_completo['delta_2'] + df_completo['delta_3']


##### Rolling skewness y kurtosis

In [37]:
df_completo['skew_6'] = df_completo['tn'].rolling(6).skew()
df_completo['kurt_6'] = df_completo['tn'].rolling(6).kurt()

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\1292573567.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['skew_6'] = df_completo['tn'].rolling(6).skew()
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\1292573567.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['kurt_6'] = df_completo['tn'].rolling(6).kurt()


##### Cambio en media móvil

In [38]:
df_completo['mean_change_6'] = df_completo['rolling_mean_6'].diff()

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\818301294.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['mean_change_6'] = df_completo['rolling_mean_6'].diff()


#####  Slope entre dos puntos (último vs. anterior)

In [39]:
df_completo['slope_last2'] = df_completo['tn'] - df_completo['lag_1']

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3090678696.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['slope_last2'] = df_completo['tn'] - df_completo['lag_1']


##### Momentum

In [40]:
df_completo['momentum_3'] = df_completo['tn'] - df_completo['lag_3']

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2347456002.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['momentum_3'] = df_completo['tn'] - df_completo['lag_3']


##### Rolling quantiles

In [41]:
df_completo['quantile_25_6'] = df_completo['tn'].rolling(6).quantile(0.25)
df_completo['quantile_75_6'] = df_completo['tn'].rolling(6).quantile(0.75)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3306008784.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['quantile_25_6'] = df_completo['tn'].rolling(6).quantile(0.25)
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3306008784.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['quantile_75_6'] = df_completo['tn'].rolling(6).quantile(0.75)


##### Categoría como frecuencia histórica

In [43]:
df_completo['freq_cat1'] = df_completo.groupby('cat1')['tn'].transform('count')

##### Cantidad de meses sin ventas en los últimos 6

In [44]:
df_completo['zeros_6'] = df_completo['tn'].rolling(6).apply(lambda x: (x == 0).sum())

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\637742352.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['zeros_6'] = df_completo['tn'].rolling(6).apply(lambda x: (x == 0).sum())


#####  Ratio de spikes

In [45]:
df_completo['spike_ratio_3'] = (df_completo['spike_up'] + df_completo['spike_down']) / 3

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2606681626.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['spike_ratio_3'] = (df_completo['spike_up'] + df_completo['spike_down']) / 3


##### Ratio entre tn y su z-score

In [46]:
df_completo['tn_zscore_ratio'] = df_completo['tn'] / (df_completo['zscore_6'] + 1e-6)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\269071183.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['tn_zscore_ratio'] = df_completo['tn'] / (df_completo['zscore_6'] + 1e-6)


##### Cambio en residual

In [47]:
df_completo['residual_change'] = df_completo['residual_decomposed'].diff()

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3206031185.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['residual_change'] = df_completo['residual_decomposed'].diff()


##### Valor relativo respecto al rango local

In [48]:
df_completo['position_in_range_6'] = (df_completo['tn'] - df_completo['rolling_min_6']) / (
    df_completo['rolling_max_6'] - df_completo['rolling_min_6'] + 1e-6)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3388763445.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_completo['position_in_range_6'] = (df_completo['tn'] - df_completo['rolling_min_6']) / (


##### Correlaciones

In [50]:
# Paso 1: Pivot para tener productos como columnas
pivot = df_completo.pivot(index='periodo', columns='product_id', values='tn')

# Paso 2: Top 15 productos más vendidos (en todo el histórico)
top_15_productos = (
    df_completo.groupby('product_id')['tn']
    .sum()
    .sort_values(ascending=False)
    .head(15)
    .index.tolist()
)

# Paso 3: Calcular la media de correlación de cada producto con los top 15
correlations = []

for prod in pivot.columns:
    if prod in top_15_productos:
        # No calculamos autocorrelación
        continue
    # Obtener serie del producto actual
    serie_prod = pivot[prod]
    
    # Calcular correlaciones con cada uno de los top 15
    corrs = []
    for top_prod in top_15_productos:
        if top_prod in pivot.columns:
            corr = serie_prod.corr(pivot[top_prod])
            if pd.notnull(corr):
                corrs.append(corr)
    if corrs:
        correlations.append((prod, np.mean(corrs)))

# Paso 4: Convertir a DataFrame y mapear
df_corrs = pd.DataFrame(correlations, columns=['product_id', 'correlacion_top15'])
df_completo = df_completo.merge(df_corrs, on='product_id', how='left')


c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


##### Completamos NaN del target con ceros

In [52]:
df_completo['target'].fillna(0, inplace=True)

C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\3329406196.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_completo['target'].fillna(0, inplace=True)


##### Dividimos el dataset

In [53]:
dt_kgl = df_completo[df_completo["periodo"].isin([201912])]
ts = df_completo.drop(df_completo[df_completo["periodo"].isin([201911,201912])].index,axis=0)

In [54]:
# Asegurate de que 'periodo' sea datetime si no lo es
# df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'].astype(str), format='%Y%m')

# features
feature_columns = [col for col in ts.columns if col not in ['periodo_dt', 'tn_target', 'nacimiento_producto', 'target']]

# Definimos los límites
train_cutoff = '2019-09-01'
valid_cutoff = '2019-11-01'

# Split temporal
df_train = df_completo[df_completo['periodo_dt'] < train_cutoff]
df_valid = df_completo[(df_completo['periodo_dt'] >= train_cutoff) & (df_completo['periodo_dt'] < valid_cutoff)]

# Separás features y target
X_train = df_train.drop(columns=feature_columns)
y_train = df_train['target']

X_valid = df_valid.drop(columns=feature_columns)
y_valid = df_valid['target']


##### Productos a predecir

In [55]:
X_kgl = dt_kgl[feature_columns]
productos_a_predecir = pd.read_csv("../../data/raw/product_id_apredecir201912.csv")
# Filtrar filas
productos_filtrados = productos_a_predecir['product_id'].unique()
X_kgl = X_kgl[X_kgl['product_id'].isin(productos_filtrados)]
X_kgl['product_id'].nunique()

780

In [ ]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35888 entries, 0 to 44387
Columns: 130 entries, product_id to lag_12_season_adj
dtypes: datetime64[ns](2), float64(108), int32(13), int64(3), object(4)
memory usage: 35.1+ MB


##### Optimización de Hiperparámetros con Optuna

In [59]:
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np

# Preparar datos
feature_columns = [col for col in ts.columns if col not in ['periodo_dt', 'tn_target', 'nacimiento_producto', 'target']]
X = ts[feature_columns]
y = ts['target']

for col in ['cat1', 'cat2', 'cat3', 'brand']:
    X[col] = X[col].astype('category')

# Eliminar NaNs en el target
if y.isnull().any():
    print("⚠️ Target tiene NaN, se eliminarán.")
    mask = ~y.isnull()
    X = X[mask]
    y = y[mask]

# Función objetivo para Optuna
def objective(trial):
    params = {
        'objective': 'regression',
        # 'tweedie_variance_power': trial.suggest_float('tweedie_variance_power', 1.1, 1.9),
        # 'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'random_state': 12345,
        'max_bin': 500,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True)
    }

    tscv = TimeSeriesSplit(n_splits=3)
    rmses = []

    for train_idx, valid_idx in tscv.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            callbacks=[
                lgb.early_stopping(50),
                lgb.log_evaluation(0)
            ]
        )

        preds = model.predict(X_valid)
        rmse = mean_squared_error(y_valid, preds, squared=False)
        rmses.append(rmse)

    return np.mean(rmses)

# Crear estudio Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Mostrar mejores parámetros
print("Mejores parámetros encontrados:", study.best_params)

# Entrenar modelo final
best_params = study.best_params
best_model = lgb.LGBMRegressor(**best_params, objective='regression', max_bin=500, random_state=12345)
best_model.fit(X, y)

print("✅ Modelo LightGBM optimizado y entrenado con éxito.")


C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2002684589.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2002684589.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')
C:\Users\iparra\AppData\Local\Temp\ipykernel_18724\2002684589.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006722 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63254
[LightGBM] [Info] Number of data points in the train set: 8357, number of used features: 150
[LightGBM] [Info] Start training from score 51.051001
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[547]	valid_0's rmse: 26.7418
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007987 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63329
[LightGBM] [Info] Number of data points in the train set: 16713, number of used features: 152
[LightGBM] [Info] Start training from score 45.167219


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[547]	valid_0's rmse: 28.589


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007741 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63329
[LightGBM] [Info] Number of data points in the train set: 16713, number of used features: 152
[LightGBM] [Info] Start training from score 45.167219
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009320 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63864
[LightGBM] [Info] Number of data points in the train set: 25069, number of used features: 153
[LightGBM] [Info] Start training from score 40.604746
Training until validation scores don't improve for 50 rounds


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-06-10 09:53:49,484] Trial 49 finished with value: 14.111929943405372 and parameters: {'num_leaves': 116, 'max_depth': 15, 'learning_rate': 0.014294906544823247, 'n_estimators': 651, 'min_child_samples': 40, 'subsample': 0.6166250564863196, 'colsample_bytree': 0.976478607628599, 'reg_alpha': 0.0002708078697149976, 'reg_lambda': 5.306969468808936e-05}. Best is trial 44 with value: 11.943959259349013.


Early stopping, best iteration is:
[318]	valid_0's rmse: 12.2256
Mejores parámetros encontrados: {'num_leaves': 69, 'max_depth': 15, 'learning_rate': 0.024669661727690494, 'n_estimators': 481, 'min_child_samples': 6, 'subsample': 0.6967056448183867, 'colsample_bytree': 0.9118594763391731, 'reg_alpha': 0.0032362427678406624, 'reg_lambda': 4.967577465599254e-06}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010257 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63899
[LightGBM] [Info] Number of data points in the train set: 33425, number of used features: 153
[LightGBM] [Info] Start training from score 37.177162
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
✅ Modelo LightGBM optimizado y entrenado con éxito.


In [60]:
X_kgl = X_kgl[feature_columns]
for col in ['cat1', 'cat2', 'cat3', 'brand']:
    X_kgl[col] = X_kgl[col].astype('category')

y_pred = best_model.predict(X_kgl)


In [61]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
result = pd.DataFrame({"product_id": X_kgl["product_id"],  "tn": y_pred})
result = result[result["product_id"].isin(productos_ok["product_id"])]
result = result.groupby("product_id").agg({"tn":"sum"}).reset_index()
result

,product_id,tn
0,20001,-7.541191
1,20002,5.919637
2,20003,1.057103
3,20004,0.387169
4,20005,0.148572
...,...,...
775,21263,0.012097
776,21265,0.012097
777,21266,0.012097
778,21267,0.012097


In [73]:
result.to_csv("../../outputs/lgb_exp04_con_bayesiana_tweeede_v3.csv", index=False, sep=',')

In [74]:
pd.set_option('display.max_rows', None)
# Supongamos que usaste un DataFrame para entrenar:
feature_names = X.columns  # Si X_train es un DataFrame

# Obtener importancias
importances = best_model.feature_importances_

# Combinar en un DataFrame para visualizar mejor
importancia_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

importancia_df


,Variable,Importancia
4,cat3,1681
5,brand,1202
0,product_id,899
7,stock_final,677
94,ytd_sum,636
6,sku_size,497
115,best_month_rank,435
82,expanding_mean,434
138,skew_6,407
83,cumulative_sum,374
